In [55]:
# Import necessary libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from scipy.stats import norm
import matplotlib.pyplot as plt
import os

import warnings
warnings.filterwarnings('ignore')

import yfinance as yf
from pandas.tseries.offsets import DateOffset
from datetime import datetime
start_date='2010-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

def snp500_tickers():
    wiki_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    ticker_df = wiki_data[0]
    ticker_list= ticker_df['Symbol'].to_list()
    # Correcting Ticker Errors in Wiki_data
    ticker_list[62]='BRK-B' 
    ticker_list[78]='BF-B'
    stock_list = ticker_df['Security'].to_list()
    stock_ticker_list = []
    stock_ticker_dict = {}
    stock_ticker_dict['S&P 500'] = '^GSPC'
    #stock_ticker_list.append(stock_ticker_dict)
    i=0
    for stock in stock_list:
        stock_ticker_dict[stock] = ticker_list[i]
        #stock_ticker_list.append(stock_ticker_dict)
        i+=1
    return(stock_ticker_dict)
# Only For Master File Generation
def stock_price_data():
    ticker_dict = snp500_tickers()
    stocks_price = extract_stock_data(ticker_dict)
    stocks_price.to_csv('./Resources/stocks_price_data.csv',index=True)
    return(stocks_price)

# Extract Stock Price for Select Stock:Ticker Pair
def extract_stock_data(ticker_dict):
    # Input dictionary with {stock_name:ticker_symbol}
    stocks_data = pd.DataFrame()
    for stock,ticker in ticker_dict.items():
        stock_df = yf.download(ticker,start_date,end_date)
        stock_df.index = pd.to_datetime(stock_df.index)
        stock_df = stock_df[['Close']]
        stock_df = stock_df.rename(columns={'Close':stock})
        stocks_data = pd.concat([stocks_data, stock_df],axis=1)

    return(stocks_data)

# Extract Stock Fundamentals Select Stock:Ticker Pair
def stock_key_financials(ticker_dict):
    stock_fundamentals = pd.DataFrame(index = list(ticker_dict))

    for stock,ticker in ticker_dict.items():
        key_info = yf.Ticker(ticker).info
        stock_fundamentals.loc[stock,'Ticker'] = ticker
        try:
            stock_fundamentals.loc[stock,'Industry'] = key_info['industry']
        except KeyError:
            pass
        try:
            stock_fundamentals.loc[stock,'Sector'] = key_info['sector']
        except KeyError:
            pass
        try:
            stock_fundamentals.loc[stock,'MarketCap(BN)'] = round(key_info['marketCap']/1000000000,2)
        except KeyError:
            pass
        stock_fundamentals.loc[stock,'SMA50'] = key_info['fiftyDayAverage']
        stock_fundamentals.loc[stock,'SMA200'] = key_info['twoHundredDayAverage']
        try:
            stock_fundamentals.loc[stock,'ProfitMargins'] = key_info['profitMargins']
        except KeyError:
            pass
        try:
            stock_fundamentals.loc[stock,'Beta'] = key_info['beta']   
        except KeyError:
            pass
        try:
            stock_fundamentals.loc[stock,'DividendYield'] = key_info['dividendYield']
        except KeyError:
            pass
        try:
            stock_fundamentals.loc[stock,'P/B_Ratio'] = key_info['priceToBook']
        except KeyError:
            pass
        try:
            stock_fundamentals.loc[stock,'PEG_Ratio'] = key_info['pegRatio']
        except KeyError:
            pass
        try:
            stock_fundamentals.loc[stock,'Debt/Equity'] = key_info['debtToEquity']
        except KeyError:
            pass
        try:
            stock_fundamentals.loc[stock,'EPS'] = key_info['trailingEps']
        except KeyError:
            pass
        try:
            stock_fundamentals.loc[stock,'NoOfOpinions'] = key_info['numberOfAnalystOpinions']
        except KeyError:
            pass
        try:
            stock_fundamentals.loc[stock,'Recommendation'] = key_info['recommendationKey']
        except KeyError:
            pass
        stock_fundamentals.loc[stock_fundamentals.index=='S&P 500','Beta'] = 1.00

    return(stock_fundamentals)

# Extracting Risk Free Rates
def risk_free_rates():
    rf_dict = {'13W_Risk_Free':'^IRX','10Y_Risk_Free':'^TNX'}
    rf_rates = pd.DataFrame()
    for key, value in rf_dict.items():
        rf_rates_df = yf.download(value,start_date,end_date)
        rf_rates_df.index = pd.to_datetime(rf_rates_df.index)
        rf_rates[key] = rf_rates_df['Close']

    return(rf_rates)

# Sharpe Ratio
def stock_sharpe_ratio(ticker_dict,investment_term_days):
    stocks_data = extract_stock_data(ticker_dict)
    rf_rate = risk_free_rates()
    if investment_term_days<= 252:
        rate = '13W_Risk_Free'
    else:
        rate = '10Y_Risk_Free'

    Sharpe_Ratio = pd.DataFrame(index=stocks_data.columns)
    #Sharpe_Ratio.index.name = 'Name'

    log_returns = np.log(stocks_data/stocks_data.shift(1))
    rolling_mean = log_returns.rolling(window=investment_term_days).mean()
    returns = np.exp(rolling_mean*investment_term_days)-1
    avg_returns = returns.mean(skipna=True)
    avg_volatility = returns.std(skipna=True)
    avg_rf_rate = rf_rate[rate]*(investment_term_days/252)/100
    sharpe_ratio = (avg_returns-avg_rf_rate.mean())/avg_volatility
    Sharpe_Ratio['avg_returns']= avg_returns
    Sharpe_Ratio['avg_volatility']=avg_volatility
    Sharpe_Ratio['Avg_Annual_Returns']= (np.exp(rolling_mean*252)-1).mean(skipna=True)
    Sharpe_Ratio['Avg_Annual_Volatility']=(np.exp(rolling_mean*252)-1).std(skipna=True)
    Sharpe_Ratio['Sharpe_Ratio']=sharpe_ratio

    #Sharpe_Ratio.to_csv('./Resources/sharpe_ratio.csv',index=True)

    return(Sharpe_Ratio)

#MCSimulation of Stock Returns
def stock_mc_simulation(ticker_dict,investment_term_days):
    stocks = extract_stock_data(ticker_dict)
    log_returns = np.log(stocks/stocks.shift(1))
    volatility = log_returns.std(skipna=True)
    rf_rate_df = risk_free_rates()
    if investment_term_days<= 252:
        rate = 0
    else:
        rate = 1
    rf_rate = round(rf_rate_df.iloc[-1,rate]/100,4)

    # Simulating Future Daily Returns
    forecast_period = investment_term_days
    simulations = 5000
    MC_Simulations_df = pd.DataFrame(index=volatility.index)
    for stock in volatility.index:
        daily_logreturns_simulated = volatility[stock] * norm.ppf(np.random.rand(forecast_period, simulations))
        daily_simplereturns_simulated = np.exp(daily_logreturns_simulated)-1
        # Calculating future price progression in each simulation
        last_price = stocks[stock].iloc[-1]
        price_list = np.zeros_like(daily_simplereturns_simulated)
        price_list[0] = last_price
        MC_Simulations_df.loc[stock,'CurrentPrice'] = round(last_price,2)
        for t in range(1, forecast_period):
            price_list[t] = price_list[t - 1] *(1+ daily_simplereturns_simulated[t])
        #Simulated Price Scenarios
        MC_Simulations_df.loc[stock,'BestPrice'] = round(price_list[-1].max(),2)
        MC_Simulations_df.loc[stock,'AvgTargetPrice'] = round(price_list[-1].mean(),2)
        #MC_Simulations_df.loc[stock,'LikelyPrice'] = statistics.mode(price_list[-1])
        MC_Simulations_df.loc[stock,'LeastPrice'] = round(price_list[-1].min(),2)
        
        #Simulated Confidence Intervals
        num_success = np.sum(price_list[-1] >= price_list[0] * (1 + target_roi))
        probability_of_success = num_success / simulations
        MC_Simulations_df.loc[stock,'%Prob of Return>=SustainableTarget'] = round(probability_of_success*100,2)

        num_loss = np.sum(price_list[-1] < price_list[0]* (1 - rf_rate))
        loss_probability = num_loss / simulations
        MC_Simulations_df.loc[stock,'%Prob of Loss>RiskFreeRate'] = round(loss_probability*100,2)
    
    return(MC_Simulations_df)

#VaR Calculations for Investments
def stock_VaR(ticker_dict,investment_amount,investment_term_days):
    inv_value = investment_amount
    rf_rate_df = risk_free_rates()
    if investment_term_days<= 252:
        rate = 0
    else:
        rate = 1
    rf_rate = round(rf_rate_df.iloc[-1,rate]/100,4)

    stocks = extract_stock_data(ticker_dict)

    log_returns = np.log(stocks/stocks.shift(1))
    rolling_mean = log_returns.rolling(window=investment_term_days).mean()
    returns = np.exp(rolling_mean*investment_term_days)-1
    vol = returns.std(skipna=True)
    
    # log_returns = np.log(stocks/stocks.shift(1))
    # STD = log_returns.std(skipna=True)
    # vol = STD*np.sqrt(investment_term_days)
    
    percentiles = [10,5,1]
    simulations = 5000
    time_period = investment_term_days/252

    VaR_df = pd.DataFrame(index=vol.index)

    for stock in vol.index:
        end_value = inv_value * np.exp((rf_rate - .5 * vol[stock] ** 2) * time_period + np.random.standard_normal(simulations) * vol[stock] * np.sqrt(time_period))
        simulated_inv_returns = end_value - inv_value
        for i in percentiles:
            value = np.percentile(simulated_inv_returns, i)
            VaR_df.loc[stock,f"VaR@{100-i}%"] = value
            #VaR_df.loc[stock,f"Percentage_VaR @{100-i}%"] = round(value*100/investment,2)

    return(VaR_df)

def stock_profile(ticker_dict,investment_amount,investment_term_days):
    stock_fundamentals_df = stock_key_financials(ticker_dict)
    sr_df = stock_sharpe_ratio(ticker_dict,investment_term_days)
    mcs_df = stock_mc_simulation(ticker_dict,investment_term_days)
    var_df = stock_VaR(ticker_dict,investment_amount,investment_term_days)
    
    stock_classifier = pd.concat([stock_fundamentals_df,mcs_df,sr_df,var_df],axis=1)

    stock_classifier['Treynor_Ratio'] = (stock_classifier['Sharpe_Ratio']*stock_classifier['avg_volatility'])/stock_classifier['Beta']

    stock_classifier["CompanySize"] = pd.cut(stock_classifier["MarketCap(BN)"],bins=[0, 0.25, 2.0, 10.0, 200.00,5000.00], labels=["Micro", "Small", "Mid", "Large","Mega"])
    # stock_preference = ['Least Preferred','Neutral','Preferred','Most Preferred']
    # stock_classifier['Return_Profile'] = pd.qcut(stock_classifier['avg_annual_returns'],4,stock_preference)
    # stock_classifier['TreynorRatio_Profile']=pd.qcut(stock_classifier['Treynor_Ratio'],4,stock_preference)
    # stock_classifier['SharpeRatio_Profile'] = pd.qcut(stock_classifier.loc[stock_classifier["sharpe_ratio"]>0,'sharpe_ratio'],4,stock_preference)
    # stock_classifier['Risk_Profile'] = pd.qcut(stock_classifier['avg_annual_volatility'],4,['Most Preferred','Preferred','Neutral','Least Preferred'])
    stock_classifier['ProfitMargin_Profile']=pd.cut(stock_classifier['ProfitMargins'],bins=[-5,0.0,0.1,0.2,5],labels=['Avoid','Low','Healthy','High'])

    info_df = stock_classifier[['Ticker','Sector','Industry','CompanySize','ProfitMargin_Profile','Recommendation']]
    numbers_df = stock_classifier[['Beta','CurrentPrice','SMA50','SMA200','Treynor_Ratio','Sharpe_Ratio']]
    predictions_df = stock_classifier[['AvgTargetPrice','%Prob of Return>=SustainableTarget',
                                         '%Prob of Loss>RiskFreeRate','VaR@99%']]
    return(info_df,numbers_df,predictions_df)

In [56]:
list1= {'3M':'MMM'}

In [57]:
#list1 = ['3M','A. O. Smith','Abbott','AbbVie','Accenture']
amount = 1000
term =252
target_roi = 0.11
info,numbers,prediction = stock_profile(list1,amount,term)
display(info)
display(numbers)
display(prediction)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Ticker,Sector,Industry,CompanySize,ProfitMargin_Profile,Recommendation
3M,MMM,Industrials,Conglomerates,Large,Avoid,hold


,Beta,CurrentPrice,SMA50,SMA200,Treynor_Ratio,Sharpe_Ratio
3M,1.044,98.93,89.76936,84.36199,0.023618,0.121204


,AvgTargetPrice,%Prob of Return>=SustainableTarget,%Prob of Loss>RiskFreeRate,VaR@99%
3M,101.36,31.22,40.56,-360.566132


In [36]:

    wiki_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    ticker_df = wiki_data[0]
    ticker_list= ticker_df['Symbol'].to_list()
    # Correcting Ticker Errors in Wiki_data
    ticker_list[62]='BRK-B' 
    ticker_list[78]='BF-B'
    stock_list = ticker_df['Security'].to_list()
    stock_ticker_df = ticker_df[['Security','Symbol']]
    stock_ticker_df = stock_ticker_df.set_index('Security')
    stock_ticker_df = stock_ticker_df.T
    stock_ticker_df['S&P 500'] = '^GSPC'
    stock_ticker_df
    
    

Security,3M,A. O. Smith,Abbott,AbbVie,Accenture,Adobe Inc.,Advanced Micro Devices,AES Corporation,Aflac,Agilent Technologies,...,Williams Companies,Willis Towers Watson,Wynn Resorts,Xcel Energy,Xylem Inc.,Yum! Brands,Zebra Technologies,Zimmer Biomet,Zoetis,S&P 500
Symbol,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,...,WMB,WTW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS,^GSPC


In [40]:


ticker_list = []
for stock in stock_list:
    print(stock_ticker_df[stock][0])
    ticker_list.append(stock_ticker_df[stock][0])



MMM
AOS
ABT
ABBV
ACN


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']